# Modeling using Regression

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modeling-using-Regression" data-toc-modified-id="Modeling-using-Regression-1">Modeling using Regression</a></span><ul class="toc-item"><li><span><a href="#Linear-Regression-All-Regions" data-toc-modified-id="Linear-Regression-All-Regions-1.1">Linear Regression All Regions</a></span></li><li><span><a href="#Linear-Regression-Individual-Regions" data-toc-modified-id="Linear-Regression-Individual-Regions-1.2">Linear Regression Individual Regions</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-1.3">Random Forest</a></span></li><li><span><a href="#Extra-Trees" data-toc-modified-id="Extra-Trees-1.4">Extra Trees</a></span></li></ul></li><li><span><a href="#Interpreting" data-toc-modified-id="Interpreting-2">Interpreting</a></span></li></ul></div>

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
import numpy as np
import datetime as dt
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches


In [6]:
from tools.tools import read_json, get_current_time

In [7]:
plt.style.use('ggplot')

In [8]:
basin_colors_hex = {  # manually defined dictionary of EIA basin-level standardized colors 
    "Anadarko Region":    "#2BA2CF", 
    "Appalachia Region":  "#769F5D",
    "Bakken Region":      "#F6C432", 
    "Eagle Ford Region":  "#48366B", 
    "Haynesville Region": "#807B8F",
    "Niobrara Region":    "#9D3341",
    "Permian Region":     "#6F4B27",
}

In [9]:
config = read_json('../config.json')

current_date = get_current_time('yyyymmdd')

wd = f"{config['workspace_directory']}/data"

In [108]:
df = pd.read_csv(f"{wd}/processing/basin_int_viirs_for_modeling.csv")

In [110]:
region_list = list(df['region'].unique())

In [11]:
df.head(3)

,year_month,region,latest_day_in_month,obs_day_cnt_avg,obs_day_cnt_med,obs_day_cnt_sum,obs_day_cnt_min,obs_day_cnt_max,qf_fit_day_avg_avg,qf_fit_day_avg_med,...,obs_day_cnt_avg_over_pct_month_completed,obs_day_cnt_med_over_pct_month_completed,obs_day_cnt_sum_over_pct_month_completed,obs_day_cnt_min_over_pct_month_completed,obs_day_cnt_max_over_pct_month_completed,obs_day_cnt_avg_per_squaremeters_over_pct_month_completed,obs_day_cnt_med_per_squaremeters_over_pct_month_completed,obs_day_cnt_sum_per_squaremeters_over_pct_month_completed,obs_day_cnt_min_per_squaremeters_over_pct_month_completed,obs_day_cnt_max_per_squaremeters_over_pct_month_completed
0,2012-03-01,Anadarko Region,2012-03-31,13.041667,12.5,313,3,33,4.786164,4.291667,...,13.041667,12.5,313.0,3.0,33.0,1.728202e-10,1.656423e-10,4.147684e-09,3.975416e-11,4.372958e-10
1,2012-03-01,Appalachia Region,2012-03-31,31.071429,32.0,870,1,86,9.502222,8.966184,...,31.071429,32.0,870.0,1.0,86.0,1.616772e-10,1.665089e-10,4.526961e-09,5.203404e-12,4.474927e-10
2,2012-03-01,Bakken Region,2012-03-31,260.321429,249.0,7289,2,571,1.497039,1.500962,...,260.321429,249.0,7289.0,2.0,571.0,2.852491e-09,2.728436e-09,7.986976e-08,2.191515e-11,6.256775e-09


In [12]:
df.shape

(679, 84)

In [200]:
X = df.drop(columns=[
    'oil_bbl_d_total_production',
    'latest_day_in_month',
    'last_day_of_month',
    'first_day_of_month',
    'year_month',
])

X = pd.get_dummies(X)

y = df['oil_bbl_d_total_production']

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.25,
    random_state=42,
)

In [201]:
X_train.shape

(509, 92)

In [202]:
for i in X.columns:
    print(i)  # check the columns putting into the model. 

obs_day_cnt_avg
obs_day_cnt_med
obs_day_cnt_sum
obs_day_cnt_min
obs_day_cnt_max
qf_fit_day_avg_avg
qf_fit_day_avg_med
qf_fit_day_avg_sum
qf_fit_day_avg_min
qf_fit_day_avg_max
qf_fit_day_med_avg
qf_fit_day_med_med
qf_fit_day_med_sum
qf_fit_day_med_min
qf_fit_day_med_max
qf_fit_day_sum_avg
qf_fit_day_sum_med
qf_fit_day_sum_sum
qf_fit_day_sum_min
qf_fit_day_sum_max
qf_fit_day_min_avg
qf_fit_day_min_med
qf_fit_day_min_sum
qf_fit_day_min_min
qf_fit_day_min_max
qf_fit_day_max_avg
qf_fit_day_max_med
qf_fit_day_max_sum
qf_fit_day_max_min
qf_fit_day_max_max
aland
obs_day_cnt_avg_per_squaremeters
obs_day_cnt_med_per_squaremeters
obs_day_cnt_sum_per_squaremeters
obs_day_cnt_min_per_squaremeters
obs_day_cnt_max_per_squaremeters
qf_fit_day_avg_avg_per_squaremeters
qf_fit_day_avg_med_per_squaremeters
qf_fit_day_avg_sum_per_squaremeters
qf_fit_day_avg_min_per_squaremeters
qf_fit_day_avg_max_per_squaremeters
qf_fit_day_med_avg_per_squaremeters
qf_fit_day_med_med_per_squaremeters
qf_fit_day_med_sum_per

In [203]:
X_test.shape

(170, 92)

## Linear Regression All Regions

In [204]:
lr = LinearRegression()
lr.fit(X_train, y_train);

In [205]:
lr.score(X_train, y_train)

0.9284652720119193

In [206]:
lr.score(X_test, y_test)

-9.437069504802942e-05

In [207]:
cross_val_score(lr, X, y, cv=10).mean()

-18.74313449830307

In [208]:
# Not good test score or cross val. Linear Regression does terribly. 
# Next try for individual regions. 

## Linear Regression Individual Regions

In [209]:
region_lr_dict = {}
for region in region_list:
    df_select = df[df['region'] == region]

    X_lr = df_select.drop(columns=[
        'oil_bbl_d_total_production',
        'latest_day_in_month',
        'last_day_of_month',
        'first_day_of_month',
        'year_month',
    ])

    X_lr = pd.get_dummies(X_lr)

    y_lr = df_select['oil_bbl_d_total_production']

    X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(
        X_lr, 
        y_lr, 
        random_state=42, 
        test_size=0.25,
    )

    lr = LinearRegression()
    lr.fit(X_train_lr, y_train_lr);

    train_score_lr = lr.score(X_train_lr, y_train_lr)

    test_score_lr = lr.score(X_test_lr, y_test_lr)
    
    cross_val_score_lr = cross_val_score(lr, X, y, cv=3).mean()
    
    region_lr_dict[region] = {
        "train_score_lr": train_score_lr,
        "test_score_lr": test_score_lr,
        "cross_val_score_lr": cross_val_score_lr,
        "model": lr,
    }
    
    print(
        region,
        "train_score_lr:", train_score_lr,
        "test_score_lr:", test_score_lr,
        "cross_val_score_lr:", cross_val_score_lr,
    ) 

Anadarko Region train_score_lr: 0.9363749376744117 test_score_lr: 0.6748751092816008 cross_val_score_lr: -5.67324886277433
Appalachia Region train_score_lr: 0.9191137204273848 test_score_lr: 0.6217166835603352 cross_val_score_lr: -5.67324886277433
Bakken Region train_score_lr: 0.8678112195215714 test_score_lr: 0.6986558870756028 cross_val_score_lr: -5.67324886277433
Eagle Ford Region train_score_lr: 0.7800431274381919 test_score_lr: -1.3345465237748777 cross_val_score_lr: -5.67324886277433
Haynesville Region train_score_lr: 0.9086195373230165 test_score_lr: 0.3727300220514495 cross_val_score_lr: -5.67324886277433
Niobrara Region train_score_lr: 0.9680902591669558 test_score_lr: -0.7017268421349283 cross_val_score_lr: -5.67324886277433
Permian Region train_score_lr: 0.9923873091090426 test_score_lr: 0.9682094672052732 cross_val_score_lr: -5.67324886277433


In [210]:
# These scores are not great either. Perhaps Tree-based. 

## Random Forest

In [211]:
rf = RandomForestRegressor(n_estimators=500, random_state=42)
rf.fit(X_train, y_train);

In [212]:
rf.score(X_train, y_train)

0.9988596379775738

In [213]:
rf.score(X_test, y_test)

0.9947515048964083

In [ ]:
cross_val_score(rf, X, y, cv=10).mean()

In [ ]:
important_features = pd.Series(data=rf.feature_importances_,index=X.columns)
important_features.sort_values(ascending=False,inplace=True)
important_features.head(20)

## Extra Trees

In [ ]:
et = ExtraTreesRegressor(random_state=42, n_estimators=500)
et.fit(X_train, y_train);

In [ ]:
et.score(X_train, y_train)

In [ ]:
et.score(X_test, y_test)

In [ ]:
cross_val_score(et, X, y, cv=10).mean()

In [ ]:
important_features = pd.Series(data=et.feature_importances_,index=X.columns)
important_features.sort_values(ascending=False,inplace=True)
important_features.head(20)

In [ ]:
y_train.shape

In [ ]:
X

In [ ]:
from sklearn.metrics import mean_squared_error

from math import sqrt

rmse = sqrt(mean_squared_error(y_test, et.predict(X_test)))

print(rmse)

272245.29239885614 / rmse

# Interpreting 

In [ ]:
predicted = cross_val_predict(et, X, y, cv=10)

In [ ]:
df['colors'] = df['region'].map(basin_colors_hex)  # adding custom colors column for plots

In [ ]:
color_labels = df['region'].unique()
# print(color_labels)
# List of RGB triplets
rgb_values = sns.color_palette("Set2", 7)

# Map label to RGB
color_map = dict(zip(color_labels, rgb_values))

plt.figure(figsize=(16, 16))
# fig, ax = plt.subplots()

plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=5, color='gray', alpha=0.75)

plt.scatter(y, predicted, c=df['colors'], edgecolors='white', s=150, alpha=1, linewidths=0.75)

plt.xlabel('Measured', fontsize=16)
plt.ylabel('Predicted', fontsize=16)

#plt.legend()
plt.show();

In [ ]:
resids = y - predicted

In [ ]:
# Normality of residuals assumption
plt.figure(figsize=(16, 16))

plt.hist(resids, bins=25)
plt.axvline(0, color='black');

In [ ]:
plt.figure(figsize=(16, 16))

plt.scatter(predicted, resids, edgecolors='black', c=df['colors'])

#legend_elements = [Patch(facecolor='orange', edgecolor='r', label='Color Patch')]
plt.legend()
plt.axhline(0, color='red');

In [ ]:
df['predicted'] = predicted

In [ ]:
#df.set_index('year_month', inplace=True)
df.sort_index(inplace=True)  # sort by datetime just to be safe

In [ ]:
def eia_variable_plot(df_in, pred_var, actual_var, legend_loc=2):
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    region_list = list(df_in['region'].unique())
    region_list.sort()

    for region in region_list:
        df_pred = df_in[df_in['region'] == region][['region', pred_var, 'colors']]
        df_pred.plot(
            label=region,
            legend=True,
            ax=ax,
            color=df_pred['colors'],
            linewidth=2.5,
            linestyle='dashed',
        )
        
        df_actual = df_in[df_in['region'] == region][['region', actual_var, 'colors']]
        df_actual.plot(
            label=region,
            legend=True,
            ax=ax,
            color=df_actual['colors'],
            linewidth=2.5, 
        )

    fmt = '{x:,.0f}'
    # tick = mtick.StrMethodFormatter(fmt)
    # ax.yaxis.set_major_formatter(tick) 
    # ax.yaxis.set_minor_formatter(tick) 

    ax.set_ylabel(pred_var.replace('_', ' ').upper())
    ax.set_xlabel("MONTHLY EIA REPORTS")
    
    plt.legend(
        title='REGIONS', 
        loc=legend_loc, 
        shadow=True, 
        facecolor='white', 
        labels=[val for val in region_list for _ in (0, 1)],  #region_list,
    )
    plt.title(pred_var.replace('_', ' ').upper())
    
    plt.savefig(f'../images/eia_report_{pred_var}.png', dpi=300)
    plt.show();

In [ ]:
pred_var = 'predicted'
actual_var = 'oil_bbl_d_total_production'
eia_variable_plot(df, pred_var, actual_var)

In [ ]:
pred_var = 'predicted'
actual_var = 'oil_bbl_d_total_production'
eia_variable_plot(df[df['region'] == 'Anadarko Region'], pred_var, actual_var)

In [ ]:
eia_variable_plot(df[df['region'] == 'Appalachia Region'], pred_var, actual_var)

In [ ]:
eia_variable_plot(df[df['region'] == "Bakken Region"], pred_var, actual_var)

In [ ]:
eia_variable_plot(df[df['region'] == "Eagle Ford Region"], pred_var, actual_var)

In [ ]:
eia_variable_plot(df[df['region'] == "Haynesville Region"], pred_var, actual_var)


In [ ]:
eia_variable_plot(df[df['region'] == "Niobrara Region"], pred_var, actual_var)


In [ ]:
eia_variable_plot(df[df['region'] == "Permian Region"], pred_var, actual_var)

In [ ]:

#eia_variable_plot(df, var)

In [ ]:
plt.figsize=(20, 10)

df[df['region'] == 'Permian Region'][['obs_day_cnt_sum_over_pct_month_completed']].plot()
df[df['region'] == 'Permian Region'][['predicted']].plot()
df[df['region'] == 'Permian Region'][['oil_bbl_d_total_production']].plot()

In [ ]:
plt.figsize=(20, 10)

df[df['region'] == 'Bakken Region'][['obs_day_cnt_sum_over_pct_month_completed']].plot()
#df[df['region'] == 'Bakken Region'][['predicted']].plot()
df[df['region'] == 'Bakken Region'][['oil_bbl_d_total_production']].plot()

In [ ]:
df[df['region'] == 'Bakken Region'][['obs_day_cnt_sum_over_pct_month_completed']].rolling(6).mean().plot()
df[df['region'] == 'Bakken Region'][['obs_day_cnt_sum_over_pct_month_completed']].rolling(3).mean().plot()

df[df['region'] == 'Bakken Region'][['obs_day_cnt_sum_over_pct_month_completed']].plot()